In [ ]:
# 수정된 쓰레기 탐지 시스템 - 정확한 바운딩 박스 처리

import torch
import cv2 as cv
from ultralytics import YOLO
import numpy as np
import os
import json
from pathlib import Path
import shutil
import random
from datetime import datetime
import yaml
import time

class FixedDataConverter:
    """수정된 데이터 변환기 - 정확한 바운딩 박스 라벨링"""
    
    def __init__(self, source_path="trashnet", target_path="dataset"):
        self.source_path = Path(source_path)
        self.target_path = Path(target_path)
        
        # 클래스 매핑
        self.class_mapping = {
            'glass': 0,
            'metal': 1,
            'paper': 2,
            'plastic': 3,
            'trash': 4
        }
        
        self.setup_yolo_structure()
    
    def setup_yolo_structure(self):
        """YOLO 폴더 구조 생성"""
        dirs = [
            self.target_path / "images" / "train",
            self.target_path / "images" / "val",
            self.target_path / "labels" / "train", 
            self.target_path / "labels" / "val"
        ]
        
        for dir_path in dirs:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    def create_smart_bounding_box(self, image, class_id):
        """스마트 바운딩 박스 생성 - 객체의 실제 영역 탐지"""
        
        # 1. 이미지를 그레이스케일로 변환
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        
        # 2. 가우시안 블러 적용
        blurred = cv.GaussianBlur(gray, (5, 5), 0)
        
        # 3. 적응형 임계값 적용
        thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                    cv.THRESH_BINARY, 11, 2)
        
        # 4. 모폴로지 연산으로 노이즈 제거
        kernel = np.ones((3, 3), np.uint8)
        cleaned = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
        cleaned = cv.morphologyEx(cleaned, cv.MORPH_OPEN, kernel)
        
        # 5. 컨투어 찾기
        contours, _ = cv.findContours(cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            # 컨투어가 없으면 전체 이미지의 80% 영역을 바운딩 박스로
            h, w = image.shape[:2]
            margin = 0.1
            return (int(w * margin), int(h * margin), 
                   int(w * (1 - margin)), int(h * (1 - margin)))
        
        # 6. 가장 큰 컨투어들 찾기 (상위 3개)
        contours = sorted(contours, key=cv.contourArea, reverse=True)[:3]
        
        # 7. 선택된 컨투어들의 바운딩 박스 계산
        all_points = []
        for contour in contours:
            area = cv.contourArea(contour)
            # 최소 면적 필터링
            if area > image.shape[0] * image.shape[1] * 0.01:  # 전체 이미지의 1% 이상
                for point in contour:
                    all_points.extend(point)
        
        if not all_points:
            # 유효한 컨투어가 없으면 중앙 80% 영역
            h, w = image.shape[:2]
            margin = 0.1
            return (int(w * margin), int(h * margin), 
                   int(w * (1 - margin)), int(h * (1 - margin)))
        
        # 8. 모든 점들을 포함하는 바운딩 박스
        all_points = np.array(all_points)
        x_min = np.min(all_points[:, 0])
        y_min = np.min(all_points[:, 1])
        x_max = np.max(all_points[:, 0])
        y_max = np.max(all_points[:, 1])
        
        # 9. 바운딩 박스를 약간 확장 (5% 마진)
        h, w = image.shape[:2]
        margin_x = int((x_max - x_min) * 0.05)
        margin_y = int((y_max - y_min) * 0.05)
        
        x_min = max(0, x_min - margin_x)
        y_min = max(0, y_min - margin_y)
        x_max = min(w, x_max + margin_x)
        y_max = min(h, y_max + margin_y)
        
        return (x_min, y_min, x_max, y_max)
    
    def process_single_image(self, img_path, class_id, split):
        """개별 이미지 처리 - 스마트 바운딩 박스 적용"""
        try:
            # 이미지 로드
            image = cv.imread(str(img_path))
            if image is None:
                print(f"⚠️ 이미지 로드 실패: {img_path}")
                return False
            
            # 새 파일명 생성
            timestamp = datetime.now().strftime('%H%M%S%f')[:8]
            new_filename = f"{class_id}_{img_path.stem}_{timestamp}.jpg"
            
            # 이미지 저장
            target_img_path = self.target_path / "images" / split / new_filename
            cv.imwrite(str(target_img_path), image)
            
            # 스마트 바운딩 박스 생성
            x_min, y_min, x_max, y_max = self.create_smart_bounding_box(image, class_id)
            
            # YOLO 형식으로 변환
            h, w = image.shape[:2]
            center_x = (x_min + x_max) / (2 * w)
            center_y = (y_min + y_max) / (2 * h)
            width = (x_max - x_min) / w
            height = (y_max - y_min) / h
            
            # 라벨 파일 저장
            label_filename = f"{new_filename.rsplit('.', 1)[0]}.txt"
            label_path = self.target_path / "labels" / split / label_filename
            
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
            
            return True
            
        except Exception as e:
            print(f"❌ 처리 실패 {img_path}: {e}")
            return False
    
    def convert_to_yolo_format(self, train_ratio=0.8):
        """데이터 변환 실행"""
        print("🔄 스마트 바운딩 박스로 변환 시작...")
        
        total_converted = 0
        train_count = 0
        val_count = 0
        
        for folder_name, class_id in self.class_mapping.items():
            folder_path = self.source_path / folder_name
            
            if not folder_path.exists():
                print(f"⚠️ 폴더 없음: {folder_name}")
                continue
            
            # 이미지 파일 찾기
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            images = []
            
            for ext in image_extensions:
                images.extend(list(folder_path.glob(f"*{ext}")))
                images.extend(list(folder_path.glob(f"*{ext.upper()}")))
            
            if not images:
                print(f"⚠️ 이미지 없음: {folder_name}")
                continue
            
            print(f"📁 {folder_name} 변환 중... ({len(images)}장)")
            
            # 랜덤 섞기 및 분할
            random.shuffle(images)
            split_idx = int(len(images) * train_ratio)
            
            train_images = images[:split_idx]
            val_images = images[split_idx:]
            
            # 변환 실행
            train_success = 0
            val_success = 0
            
            for img_path in train_images:
                if self.process_single_image(img_path, class_id, "train"):
                    train_success += 1
            
            for img_path in val_images:
                if self.process_single_image(img_path, class_id, "val"):
                    val_success += 1
            
            train_count += train_success
            val_count += val_success
            total_converted += train_success + val_success
            
            print(f"✅ {folder_name}: {train_success}장(train) + {val_success}장(val)")
        
        print(f"\n🎉 변환 완료! 총 {total_converted}장")
        return train_count, val_count


class ImprovedInteractiveLabeler:
    """개선된 대화형 라벨러"""
    
    def __init__(self, dataset_path="dataset"):
        self.dataset_path = Path(dataset_path)
        self.current_image = None
        self.current_image_path = None
        self.current_label_path = None
        self.bboxes = []
        self.drawing = False
        self.start_point = None
        self.current_class = 0
        
        # 이미지 목록
        self.image_files = list((self.dataset_path / "images" / "train").glob("*.jpg"))
        self.current_index = 0
        
        # 클래스 정보
        self.class_names = ['glass', 'metal', 'paper', 'plastic', 'trash']
        self.class_colors = [
            (0, 255, 255),   # glass - yellow
            (255, 0, 255),   # metal - magenta
            (255, 255, 0),   # paper - cyan
            (0, 255, 0),     # plastic - green
            (0, 0, 255)      # trash - red
        ]
        
        print(f"📝 {len(self.image_files)}개 이미지 정밀 라벨링 준비")
    
    def mouse_callback(self, event, x, y, flags, param):
        """개선된 마우스 콜백"""
        if event == cv.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        
        elif event == cv.EVENT_MOUSEMOVE:
            if self.drawing and self.start_point:
                # 실시간 박스 미리보기
                img_preview = self.current_image.copy()
                self.draw_all_boxes(img_preview)
                
                # 현재 그리는 박스
                color = self.class_colors[self.current_class]
                cv.rectangle(img_preview, self.start_point, (x, y), color, 2)
                cv.imshow('Interactive Labeler', img_preview)
        
        elif event == cv.EVENT_LBUTTONUP:
            if self.drawing and self.start_point:
                self.drawing = False
                end_point = (x, y)
                
                # 박스 크기 검증
                x1, y1 = self.start_point
                x2, y2 = end_point
                
                if abs(x2 - x1) < 10 or abs(y2 - y1) < 10:
                    print("⚠️ 박스가 너무 작습니다.")
                    return
                
                # 좌표 정렬
                if x1 > x2:
                    x1, x2 = x2, x1
                if y1 > y2:
                    y1, y2 = y2, y1
                
                bbox = (self.current_class, x1, y1, x2, y2)
                self.bboxes.append(bbox)
                
                print(f"✅ 박스 추가: {self.class_names[self.current_class]} ({x1}, {y1}) to ({x2}, {y2})")
                self.draw_current_state()
    
    def draw_all_boxes(self, img):
        """모든 박스 그리기"""
        for bbox in self.bboxes:
            class_id, x1, y1, x2, y2 = bbox
            color = self.class_colors[class_id]
            
            # 박스 그리기
            cv.rectangle(img, (x1, y1), (x2, y2), color, 2)
            
            # 라벨 배경
            label = f"{self.class_names[class_id]}"
            (text_w, text_h), _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv.rectangle(img, (x1, y1 - text_h - 5), (x1 + text_w, y1), color, -1)
            
            # 라벨 텍스트
            cv.putText(img, label, (x1, y1 - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    def draw_current_state(self):
        """현재 상태 화면에 그리기"""
        if self.current_image is None:
            return
        
        img_display = self.current_image.copy()
        self.draw_all_boxes(img_display)
        
        # 상단 정보 배경
        overlay = img_display.copy()
        cv.rectangle(overlay, (0, 0), (img_display.shape[1], 120), (0, 0, 0), -1)
        cv.addWeighted(overlay, 0.7, img_display, 0.3, 0, img_display)
        
        # 정보 표시
        info = f"[{self.current_index + 1}/{len(self.image_files)}] Boxes: {len(self.bboxes)}"
        cv.putText(img_display, info, (10, 25), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # 현재 선택된 클래스 표시
        current_class_text = f"Current: {self.class_names[self.current_class]}"
        current_color = self.class_colors[self.current_class]
        cv.putText(img_display, current_class_text, (10, 55), cv.FONT_HERSHEY_SIMPLEX, 0.8, current_color, 2)
        
        # 조작법
        controls = [
            "Mouse: Draw | SPACE: Save | N: Next | P: Prev",
            "C: Clear | 0-4: Class | U: Undo | Q: Quit"
        ]
        for i, control in enumerate(controls):
            cv.putText(img_display, control, (10, 85 + i * 25), 
                      cv.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
        
        cv.imshow('Interactive Labeler', img_display)
    
    def save_labels(self):
        """라벨 저장"""
        if not self.bboxes:
            # 빈 라벨 파일 생성
            with open(self.current_label_path, 'w') as f:
                pass
            print("💾 빈 라벨 파일 저장")
            return
        
        h, w = self.current_image.shape[:2]
        
        with open(self.current_label_path, 'w') as f:
            for bbox in self.bboxes:
                class_id, x1, y1, x2, y2 = bbox
                
                # YOLO 형식으로 변환
                center_x = (x1 + x2) / (2 * w)
                center_y = (y1 + y2) / (2 * h)
                width = abs(x2 - x1) / w
                height = abs(y2 - y1) / h
                
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
        
        print(f"💾 저장 완료: {len(self.bboxes)}개 박스")
    
    def load_current_image(self):
        """현재 이미지 로드"""
        if self.current_index >= len(self.image_files):
            return
        
        self.current_image_path = self.image_files[self.current_index]
        self.current_image = cv.imread(str(self.current_image_path))
        
        # 라벨 파일 경로
        label_name = self.current_image_path.stem + ".txt"
        self.current_label_path = self.dataset_path / "labels" / "train" / label_name
        
        # 기존 라벨 로드
        self.load_existing_labels()
        
        # 현재 클래스를 파일명에서 추출
        try:
            self.current_class = int(self.current_image_path.name.split('_')[0])
        except:
            self.current_class = 0
        
        self.draw_current_state()
        print(f"📁 로드: {self.current_image_path.name}")
    
    def load_existing_labels(self):
        """기존 라벨 로드"""
        self.bboxes = []
        
        if not self.current_label_path.exists():
            return
        
        h, w = self.current_image.shape[:2]
        
        try:
            with open(self.current_label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id = int(parts[0])
                        center_x = float(parts[1]) * w
                        center_y = float(parts[2]) * h
                        width = float(parts[3]) * w
                        height = float(parts[4]) * h
                        
                        x1 = int(center_x - width/2)
                        y1 = int(center_y - height/2)
                        x2 = int(center_x + width/2)
                        y2 = int(center_y + height/2)
                        
                        self.bboxes.append((class_id, x1, y1, x2, y2))
        except Exception as e:
            print(f"⚠️ 라벨 로드 실패: {e}")
    
    def start_labeling(self):
        """라벨링 시작"""
        if not self.image_files:
            print("❌ 라벨링할 이미지가 없습니다.")
            return
        
        print("\n🎯 정밀 라벨링 모드")
        print("=" * 40)
        print("📝 클래스 번호:")
        for i, name in enumerate(self.class_names):
            print(f"  {i}: {name}")
        print("=" * 40)
        
        cv.namedWindow('Interactive Labeler')
        cv.setMouseCallback('Interactive Labeler', self.mouse_callback)
        
        self.load_current_image()
        
        while True:
            key = cv.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord(' '):  # 저장
                self.save_labels()
            elif key == ord('n'):  # 다음
                self.current_index = (self.current_index + 1) % len(self.image_files)
                self.load_current_image()
            elif key == ord('p'):  # 이전
                self.current_index = (self.current_index - 1) % len(self.image_files)
                self.load_current_image()
            elif key == ord('c'):  # 클리어
                self.bboxes = []
                self.draw_current_state()
                print("🗑️ 모든 박스 삭제")
            elif key == ord('u'):  # 언두
                if self.bboxes:
                    removed = self.bboxes.pop()
                    self.draw_current_state()
                    print(f"↶ 마지막 박스 제거: {self.class_names[removed[0]]}")
            elif key >= ord('0') and key <= ord('4'):  # 클래스 선택
                self.current_class = key - ord('0')
                self.draw_current_state()
                print(f"🎯 클래스 변경: {self.class_names[self.current_class]}")
        
        cv.destroyAllWindows()
        print("✅ 정밀 라벨링 완료!")


def create_dataset_yaml():
    """데이터셋 설정 파일 생성"""
    yaml_content = """# Improved Trash Classification Dataset
path: ./dataset
train: images/train
val: images/val

# Classes
nc: 5
names: ['glass', 'metal', 'paper', 'plastic', 'trash']

# Training hyperparameters
imgsz: 640
batch_size: 8
epochs: 100
lr0: 0.01
patience: 50
"""
    
    with open("dataset.yaml", "w", encoding="utf-8") as f:
        f.write(yaml_content)
    
    print("✅ 개선된 dataset.yaml 파일 생성 완료")


def main():
    """메인 실행 함수"""
    print("🔧 수정된 쓰레기 분류 시스템")
    print("=" * 50)
    
    while True:
        print("\n다음 중 선택하세요:")
        print("1️⃣ 스마트 바운딩 박스로 데이터 변환")
        print("2️⃣ 정밀 대화형 라벨링")
        print("3️⃣ dataset.yaml 생성")
        print("4️⃣ 데이터 검증")
        print("0️⃣ 종료")
        
        choice = input("선택 (0-4): ").strip()
        
        if choice == "1":
            source_folder = input("원본 폴더 경로 (기본값: trashnet): ").strip()
            if not source_folder:
                source_folder = "trashnet"
            
            converter = FixedDataConverter(source_folder)
            train_count, val_count = converter.convert_to_yolo_format()
            
            print(f"\n✅ 변환 완료!")
            print(f"🏋️ 훈련: {train_count}장")
            print(f"🔍 검증: {val_count}장")
        
        elif choice == "2":
            labeler = ImprovedInteractiveLabeler()
            labeler.start_labeling()
        
        elif choice == "3":
            create_dataset_yaml()
        
        elif choice == "4":
            # 간단한 데이터 검증
            dataset_path = Path("dataset")
            if dataset_path.exists():
                train_imgs = len(list((dataset_path / "images" / "train").glob("*.jpg")))
                val_imgs = len(list((dataset_path / "images" / "val").glob("*.jpg")))
                train_labels = len(list((dataset_path / "labels" / "train").glob("*.txt")))
                val_labels = len(list((dataset_path / "labels" / "val").glob("*.txt")))
                
                print(f"\n📊 데이터셋 현황:")
                print(f"🏋️ 훈련 이미지: {train_imgs}장, 라벨: {train_labels}개")
                print(f"🔍 검증 이미지: {val_imgs}장, 라벨: {val_labels}개")
                
                if train_imgs != train_labels:
                    print("⚠️ 훈련 데이터: 이미지와 라벨 수가 다릅니다!")
                if val_imgs != val_labels:
                    print("⚠️ 검증 데이터: 이미지와 라벨 수가 다릅니다!")
                
                if train_imgs == train_labels and val_imgs == val_labels:
                    print("✅ 데이터셋 검증 통과!")
            else:
                print("❌ 데이터셋 폴더가 없습니다.")
        
        elif choice == "0":
            print("👋 프로그램을 종료합니다.")
            break
        
        else:
            print("❌ 잘못된 선택입니다.")


if __name__ == "__main__":
    main()